In [1]:
import itertools
import os
import os.path as osp
import pickle
import urllib
from collections import namedtuple

import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.nn import Linear 

from matplotlib.ticker import MultipleLocator, FormatStrFormatter
import pandas as pd
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import normalized_mutual_info_score
# import random
from scipy.stats import hypergeom

%matplotlib inline


In [2]:
def tensor_from_numpy(x, device):
    return torch.from_numpy(x).to(device)

def plot_loss_with_acc(loss_history, val_acc_history):
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax1.plot(range(len(loss_history)), loss_history,
             c=np.array([255, 71, 90]) / 255.)
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.title('Training Loss ')
    plt.show() 

In [3]:
# 定义模型
class HalfAutoEncoder(nn.Module):
    def __init__(self,linear4,linear5,linear6 ):
        super(HalfAutoEncoder,self).__init__()
        
        self.linear4= linear4
        self.linear5= linear5
        self.linear6= linear6

    
    def forward(self, h3):
        self.h4  = F.relu(self.linear4(h3))
        self.h4 = F.normalize(self.h4)
        
        self.h5  = F.relu(self.linear5(self.h4))
        self.h5 = F.normalize(self.h5)
        
        self.h6  = self.linear6(self.h5)

        return  self.h6

# class HalfAutoEncoder(nn.Module):
#     def __init__(self,linear5,linear6,linear7,linear8 ):
#         super(HalfAutoEncoder,self).__init__()
         
#         self.linear5= linear5
#         self.linear6= linear6
#         self.linear7= linear7
#         self.linear8= linear8
    
#     def forward(self, h4):
#         self.h5  = F.relu(self.linear5( h4))
#         self.h5 = F.normalize(self.h5)
        
#         self.h6  = F.sigmoid(self.linear6( self.h5))
#         self.h6 = F.normalize(self.h6)
        
#         self.h7  = F.relu(self.linear7(self.h6))
#         self.h7 = F.normalize(self.h7)
        
#         self.h8  = self.linear8(self.h7)

#         return  self.h8

In [4]:
# 超参数定义
# LEARNING_RATE = 1e-3 #0.001
# WEIGHT_DACAY = 1e-4
# EPOCHS = 80
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
feature_np = np.load("../model/feacture_np.npy")
 
tensor_x = tensor_from_numpy(feature_np , DEVICE)

model = torch.load( '../model/half_auto_encoder.pt', map_location=torch.device('cpu'))

labelList=pd.read_csv('../data/label.csv',header=None)[1]
for i in range(0,len(labelList)):
    labelList[i]+=1
   
tensor_y = tensor_from_numpy(np.load("../model/rec_X.npy"), DEVICE)

In [7]:
class ZeroOneEncoder():
    def __init__(self,model,tensor_x, tensor_y):
        self.model=model
        self.tensor_x=tensor_x
        self.tensor_mask=tensor_x
        self.tensor_y=tensor_y
        self.zero_one_feature=np.zeros(shape=tensor_x.shape)
        self.tensor_train_mask=[False  for i in range(0,850)]
        self.criterion=nn.MSELoss(reduction='mean')
    
    def run(self):
#         for i in range(0,tensor_x.shape[0]):
        for i in range(0,200):
            
            self.tensor_train_mask[i]=True # 一次只取一条数据出来计算loss
#             logits = self.model(self.tensor_x)  # 前向传播
#             train_mask_logits = logits[self.tensor_train_mask]   # 只选择训练节点进行监督
#             loss = self.criterion(train_mask_logits, tensor_y[self.tensor_train_mask])   #计算每一条数据的loss
            
            
            # 对每一条数据的特征一个个进行mask,看看哪个特征被mask以后引起loss剧烈变化,就置为1, 否则置为0
            loss_hist=[]
            for j in range(0,self.tensor_x.shape[1]):
                 
                self.tensor_mask[i][j]=0 #对第i条数据的第j 个特征进行 mask
                logits = self.model(self.tensor_mask)  # 前向传播
                train_mask_logits = logits[self.tensor_train_mask]   # 只选择训练节点进行监督
                mask_loss = self.criterion(train_mask_logits, tensor_y[self.tensor_train_mask])   #计算每一条数据的loss
#                 loss_hist.append( abs(mask_loss.item()-loss.item())/loss.item() )
                loss_hist.append( mask_loss.item() )
                self.tensor_mask[i][j]=self.tensor_x[i][j] #把被mask的地方还原回来
             
            thresh = np.percentile(loss_hist, 80)

#             print(loss_hist)
            for j in range(0,self.tensor_x.shape[1]):
                if loss_hist[j] > thresh:
                    self.zero_one_feature[i][j]=1
                    
            self.tensor_train_mask[i]=False

            
        return self.zero_one_feature
    

In [8]:
zero_one_encoder=ZeroOneEncoder(model,tensor_x, tensor_y)

In [9]:
zero_one_feature=zero_one_encoder.run()

In [ ]:
# np.save("../model/zero_one_feature.npy",zero_one_feature )

In [ ]:
np.where(np.array(labelList)==4) 

In [10]:
# 0
print( zero_one_feature[46: 47] )
print( zero_one_feature[64: 65] )
print( zero_one_feature[75: 76] )
print( zero_one_feature[81: 82] )
print( zero_one_feature[86: 87] )
print( zero_one_feature[93: 94] )
print( zero_one_feature[103: 104] )

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1.
  1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
  1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
  1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0.
  0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]]


In [11]:
# 1
print( zero_one_feature[0: 1] )
print( zero_one_feature[26: 27] )
print( zero_one_feature[30: 31] )
print( zero_one_feature[32: 33] )
print( zero_one_feature[39: 40] )
print( zero_one_feature[40: 41] )
print( zero_one_feature[49: 50] )

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
  0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
  0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
  0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
  0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.
  0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]]


In [12]:
 # 2
print( zero_one_feature[1: 2] )
print( zero_one_feature[5: 6] )
print( zero_one_feature[7: 8] )
print( zero_one_feature[8: 9] )
# print( zero_one_feature[12: 13] )
# print( zero_one_feature[15: 16] )
# print( zero_one_feature[16: 17] )

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.
  0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [13]:
# 3 33,  37,  52, 100, 112, 133, 150
print( zero_one_feature[33: 34] )
print( zero_one_feature[37: 38] )
print( zero_one_feature[52: 53] )
print( zero_one_feature[100: 101] )
print( zero_one_feature[112: 113] )
print( zero_one_feature[133: 134] )
print( zero_one_feature[150: 151] )

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
  1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
  1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.
  0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1.
  1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [14]:
# 4   2,  3,  27,  31,  45,  51, 62
print( zero_one_feature[2: 3] )
print( zero_one_feature[3: 4] )
print( zero_one_feature[27: 28] )
print( zero_one_feature[31: 32] )
print( zero_one_feature[45: 46] )
print( zero_one_feature[51: 52] )
print( zero_one_feature[62: 63] )

[[0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
[[0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
  1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.]]


In [15]:
zero_one_feature = np.load("../model/zero_one_feature.npy")


In [ ]:
feature_np=zero_one_feature[0:200]
# feature_np/=feature_np.sum(1, keepdims=True)
kmeans = KMeans(n_clusters=12,init='k-means++').fit(feature_np)
def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix)

print('purity : ',purity_score(labelList[0:200],kmeans.labels_ ))
# print('ARI : ',adjusted_rand_score(labelList,kmeans.labels_))
# print('NMI : ',normalized_mutual_info_score(labelList,kmeans.labels_ ))

In [ ]:
import scipy.stats as stats
from tqdm import tqdm

# 利用 超几何分布的 生存函数计算 相似度

num_pairs = int(zero_one_feature.shape[0]*(zero_one_feature.shape[0]-1)/2)


ab_vec = np.zeros((zero_one_feature.shape[0],  ))  #记录特征为1 的个数
activate_dict = {}# 储存为 特征矩阵 为1 的索引
 
for i in range(zero_one_feature.shape[0]):
    ab_vec[i] = np.sum(zero_one_feature[i])
    activate_dict[i] = np.where(zero_one_feature[i] == 1)
 

e = np.log10(num_pairs)


abc_dict = {}
def sf(a, b, c):
    key = (a, b, c)
    if not key in abc_dict:
#         val = -np.log10(stats.hypergeom.sf(c - 1, zero_one_feature.shape[1], a, b)) - e
 
        val = stats.hypergeom.pmf(c , zero_one_feature.shape[1], a, b)
 
        abc_dict[key] = val
        abc_dict[(b, a, c)] = val
    else:
        val = abc_dict[key]
    return val

np_adj=np.zeros(shape=[zero_one_feature.shape[0],zero_one_feature.shape[0]])

with tqdm(total=num_pairs) as pbar:
    for i in range(zero_one_feature.shape[0]):
        for j in range(i+1, zero_one_feature.shape[0]):
            pbar.update(1)
            a = ab_vec[i]#为1 的个数
            b = ab_vec[j]#为1 的个数
            c = np.intersect1d(activate_dict[i], activate_dict[j]).shape[0] # 都为1 的索引相同的个数

            similar= stats.hypergeom.pmf(c , zero_one_feature.shape[1], a, b)
#             similar= sf(a,b,c)
#             if similar>0:
            np_adj[i][j]=similar

 
    

In [ ]:
for i in range(1,zero_one_feature.shape[0]):
    for j in range(0,i):

        np_adj[i][j]=np_adj[j][i]

In [ ]:
            
np_adj/=np_adj.sum(1, keepdims=True)  
        

 

In [ ]:
import markov_clustering as mc

matrix = sp.csr_matrix(np.matrix(np_adj))

result = mc.run_mcl(matrix)
clusters = mc.get_clusters(result) 
Q = mc.modularity(matrix=result, clusters=clusters)
# print("modularity:", Q)

In [ ]:
len(clusters)

In [ ]:
from scipy.stats import hypergeom
import matplotlib.pyplot as plt 

[M, n, N] = [20, 7, 12]
rv = hypergeom(M, n, N)
x = np.arange(0, n+1)
pmf_dogs = rv.pmf(x)


fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x, pmf_dogs, 'bo')
ax.vlines(x, 0, pmf_dogs, lw=2)
ax.set_xlabel('# of dogs in our group of chosen animals')
ax.set_ylabel('hypergeom PMF')
plt.show()

In [ ]:
from torch.utils import data
from torch.utils.data import DataLoader, TensorDataset
class Contig_data(data.Dataset):
    def __init__(self, file, type='train'):
        self.type = type
        self.file = file
        # The last column is label, the other columns are the features
        df = pd.read_csv(self.file, header = None)
        self.labels = df.values[:, -1]
        # self.seqs = df.values[:, :-1].astype(np.float32)
        self.seqs = df.values[:, : -1].astype(np.float32)

    def __len__(self):
        return self.seqs.shape[0]

    def __getitem__(self, index):
        return self.seqs[index], self.labels[index]
    
test_data = Contig_data(file='../data/xubo_kmer/kmer.csv')
test_loader = DataLoader(test_data )
for batch_idx, (data, _) in enumerate(test_loader):
    print(data.shape)

In [ ]:
thresh = np.percentile([1,1,1,1,2], 90)

In [ ]:
thresh